In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import warnings
import librosa
from librosa import feature
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, f1_score, recall_score, accuracy_score
import pickle
warnings.filterwarnings('ignore')
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
import librosa.display
from sklearn.model_selection import train_test_split
import glob
import os
import re

## Data Collection

In [11]:
data_dir = "F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1"
audio_files = glob.glob(os.path.join(data_dir , '*.wav'))
print(len(audio_files))

920


## Features Extractions 

In [12]:
fn_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff]
 
fn_list_ii = [
 feature.rms,
 feature.zero_crossing_rate]


pattern = re.compile(r'\d\d\d')

def get_feature_vector(file,y,sr): 
    id1 = []
    a = pattern.search(file)
    id1.append(a.group())
    feat_vect_i = [np.mean(funct(y,sr)) for funct in fn_list_i]
    feat_vect_ii = [np.mean(funct(y)) for funct in fn_list_ii] 
    feature_vector = id1 + feat_vect_i + feat_vect_ii #list concatanation
    return feature_vector


In [13]:
norm_audios_feat = []
for file in audio_files:   
    print(file)
    y,sr = librosa.load(file, sr=None)    #y = time-series array , sr = sampling rate
    feature_vector = get_feature_vector(file, y, sr)
    norm_audios_feat.append(feature_vector)

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\101_1b1_Al_sc_Meditron.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\101_1b1_Pr_sc_Meditron.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\102_1b1_Ar_sc_Meditron.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\103_2b2_Ar_mc_LittC2SE.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Al_sc_Litt3200.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Ar_sc_Litt3200.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Ll_sc_Litt3200.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Lr_sc_Litt3200.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Pl_sc_Litt3200.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\104_1b1_Pr_sc_Litt3200.wav
F:/dataset

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p2_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p2_Ar_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p2_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p2_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p2_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p3_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p3_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p3_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p3_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\130_3p4_Al_mc_AKGC417L.wav
F:/dataset

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_2p4_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Ar_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Lr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p2_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\151_3p3_Ll_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\152_1b1_Al_sc_Meditron.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\153_1b1_Al_sc_Meditron.wav
F:/dataset

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b2_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b2_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Ar_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Ll_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Lr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b3_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\170_1b4_Al_mc_AKGC417L.wav
F:/dataset

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b2_Lr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b2_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b2_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b2_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Ar_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Lr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\186_2b3_Tc_mc_AKGC417L.wav
F:/dataset

F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b2_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b2_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b2_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b3_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b3_Ar_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b3_Pl_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b3_Pr_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b3_Tc_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b4_Al_mc_AKGC417L.wav
F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1\207_2b4_Ar_mc_AKGC417L.wav
F:/dataset

In [14]:
import csv

audio_file = 'F:\\dataset\\Respiratory_Sound_Database\\Respiratory_Sound_Database\\audio_features.csv'
header = ['id','chroma_stft', 'spectral_centroid','spectral_bandwidth','spectral_rolloff','rmse','zero_crossing_rate']

with open (audio_file,'w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(norm_audios_feat)


In [15]:
audio = pd.read_csv(audio_file)
audio = audio.groupby('id',as_index=False).agg('mean')
audio.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate
0,101,0.788483,106.341210,478.175310,88.115266,0.046111,0.002700
1,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014
2,103,0.776724,104.833348,562.750815,38.039137,0.255904,0.001465
3,104,0.627749,108.450674,153.273541,210.582248,0.058936,0.007269
4,105,0.579455,792.488395,971.752301,1256.439696,0.029107,0.069132


## MFCC feature extraction

In [16]:
mfcc_vector = []
for file in audio_files:
    id1 = []
    file_name = file.split("\\")[-1]
    id1.append(file_name.split("_")[0])
    y,sr = librosa.load(file,sr=16000)
    mfcc = librosa.feature.mfcc(y,sr,n_mfcc=13).T.tolist()
    mfcc_1 = np.mean(mfcc, axis=0).tolist()
    mfcc_vector.append(id1 + mfcc_1)

In [17]:
import csv

norm_output = 'F:\dataset\Respiratory_Sound_Database\Respiratory_Sound_Database\mfcc_features.csv'
header = ['id']
for i in range(1,14):
    header.append('mfcc - ' + str(i))

with open (norm_output,'w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(mfcc_vector)

In [18]:
mfcc = pd.read_csv(norm_output)
mfcc = mfcc.groupby('id',as_index=False).agg('mean')
mfcc.head()

,id,mfcc - 1,mfcc - 2,mfcc - 3,mfcc - 4,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13
0,101,-544.208437,108.624587,63.630197,36.600461,33.532526,32.800587,24.612642,17.876389,17.683792,16.083936,9.403939,4.995627,7.577631
1,102,-580.988766,121.589029,56.972991,25.953187,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239
2,103,-408.989858,71.455142,48.836396,34.063806,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760
3,104,-439.710849,97.601193,69.050068,39.048891,18.613349,9.490852,7.909424,10.076911,13.689739,16.820580,17.742319,15.963111,12.430860
4,105,-339.230309,190.384630,-15.817387,-46.475974,17.833985,19.237400,4.010118,1.151154,1.593777,3.363842,7.783992,11.781842,2.682762


## Merge both feature file

In [19]:
final_Df = audio.merge(mfcc, on='id')
final_Df.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc - 1,mfcc - 2,mfcc - 3,mfcc - 4,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13
0,101,0.788483,106.341210,478.175310,88.115266,0.046111,0.002700,-544.208437,108.624587,63.630197,36.600461,33.532526,32.800587,24.612642,17.876389,17.683792,16.083936,9.403939,4.995627,7.577631
1,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014,-580.988766,121.589029,56.972991,25.953187,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239
2,103,0.776724,104.833348,562.750815,38.039137,0.255904,0.001465,-408.989858,71.455142,48.836396,34.063806,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760
3,104,0.627749,108.450674,153.273541,210.582248,0.058936,0.007269,-439.710849,97.601193,69.050068,39.048891,18.613349,9.490852,7.909424,10.076911,13.689739,16.820580,17.742319,15.963111,12.430860
4,105,0.579455,792.488395,971.752301,1256.439696,0.029107,0.069132,-339.230309,190.384630,-15.817387,-46.475974,17.833985,19.237400,4.010118,1.151154,1.593777,3.363842,7.783992,11.781842,2.682762


In [20]:
#merge label files
df1 = pd.read_csv('F:\dataset\Respiratory_Sound_Database\Respiratory_Sound_Database\patient_diagnosis.csv',header=None)
df1.columns = ['id','label']
df1.head()

,id,label
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


## Final dataframe

In [21]:
df = final_Df.merge(df1, on='id')
df.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc - 1,mfcc - 2,mfcc - 3,...,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13,label
0,101,0.788483,106.341210,478.175310,88.115266,0.046111,0.002700,-544.208437,108.624587,63.630197,...,33.532526,32.800587,24.612642,17.876389,17.683792,16.083936,9.403939,4.995627,7.577631,URTI
1,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014,-580.988766,121.589029,56.972991,...,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239,Healthy
2,103,0.776724,104.833348,562.750815,38.039137,0.255904,0.001465,-408.989858,71.455142,48.836396,...,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760,Asthma
3,104,0.627749,108.450674,153.273541,210.582248,0.058936,0.007269,-439.710849,97.601193,69.050068,...,18.613349,9.490852,7.909424,10.076911,13.689739,16.820580,17.742319,15.963111,12.430860,COPD
4,105,0.579455,792.488395,971.752301,1256.439696,0.029107,0.069132,-339.230309,190.384630,-15.817387,...,17.833985,19.237400,4.010118,1.151154,1.593777,3.363842,7.783992,11.781842,2.682762,URTI


In [22]:
df['label'].value_counts()

COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Pneumonia          6
Bronchiolitis      6
LRTI               2
Asthma             1
Name: label, dtype: int64

In [23]:
##  Remove LRTI and Asthma because it has very less samples
df = df[df['label'] != 'LRTI']
df = df[df['label'] != 'Asthma']

In [24]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1:]
ld = LabelEncoder()
y_encode = ld.fit_transform(y)
ld.inverse_transform([0,1,2,3,4,5])

array(['Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy', 'Pneumonia',
       'URTI'], dtype=object)

In [25]:
## After removing LRTI and Asthma
df['label'].value_counts()

COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Pneumonia          6
Bronchiolitis      6
Name: label, dtype: int64

## Model Creation

### 1. Random Forest

In [30]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1:]
oversample = SMOTE()
X_ROS, y_ROS = oversample.fit_resample(X,y)
y_encode = LabelEncoder().fit_transform(y_ROS)

x_train, x_test, y_train, y_test = train_test_split(X_ROS, y_encode,test_size=0.20)

cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state = 42)
rf_classifier = RandomForestClassifier()
n_estimators = [10, 20, 30, 50, 100]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

rfgrid = GridSearchCV(rf_classifier, hyperF, cv = cv, n_jobs = -1)
rfgrid.fit(x_train, y_train)
rf_pred_probs = rfgrid.predict(x_test)
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, rf_pred_probs, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, rf_pred_probs, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, rf_pred_probs)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, rf_pred_probs)))
print('5. Accuracy \n {} \n'.format(accuracy_score(y_test, rf_pred_probs)))

filename = 'respiratory_without_noise_results\\rfgrid_model.pkl'
with open(filename, 'wb') as file:  
    pickle.dump(rfgrid, file)


1. The F-1 score of the model 0.8294828437885942

2. The recall score of the model 0.8505952380952381

3. Classification report 
               precision    recall  f1-score   support

           0       0.83      1.00      0.91        15
           1       0.92      1.00      0.96        11
           2       1.00      0.43      0.60        14
           3       0.93      0.88      0.90        16
           4       0.73      1.00      0.85        11
           5       0.73      0.80      0.76        10

    accuracy                           0.84        77
   macro avg       0.86      0.85      0.83        77
weighted avg       0.87      0.84      0.83        77
 

4. Confusion matrix 
 [[15  0  0  0  0  0]
 [ 0 11  0  0  0  0]
 [ 3  0  6  1  3  1]
 [ 0  0  0 14  0  2]
 [ 0  0  0  0 11  0]
 [ 0  1  0  0  1  8]] 

5. Accuracy 
 0.8441558441558441 



In [38]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1:]
oversample = SMOTE()
X_ROS, y_ROS = oversample.fit_resample(X,y)
y_encode = LabelEncoder().fit_transform(y_ROS)
cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)

x_train,x_test,y_train,y_test = train_test_split(X_ROS, y_encode,test_size=0.20)

knn = KNeighborsClassifier()
# knn.fit(x_train, y_train) 
# pred_probs = knn.predict(x_test)
# accuracy = accuracy_score(y_test,pred_probs)
# print("Test accuracy using KNN",accuracy)
# train_probs = knn.predict(x_train)
# accuracy = accuracy_score(y_train,train_probs)
# print("Train accuracy using KNN",accuracy)
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
knngrid = GridSearchCV(knn, hyperparameters, cv=cv)
knngrid.fit(x_train, y_train)
knn_pred_probs = knngrid.predict(x_test)
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, knn_pred_probs, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, knn_pred_probs, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, knn_pred_probs)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, knn_pred_probs)))
print('5. Accuracy \n {} \n'.format(accuracy_score(y_test, knn_pred_probs)))

filename = 'respiratory_without_noise_results\\knn_model.pkl'
with open(filename, 'wb') as file:  
    pickle.dump(knngrid, file)


1. The F-1 score of the model 0.947979797979798

2. The recall score of the model 0.944128787878788

3. Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00         9
           2       1.00      0.82      0.90        11
           3       0.91      0.91      0.91        11
           4       0.94      1.00      0.97        16
           5       0.88      0.94      0.91        16

    accuracy                           0.95        77
   macro avg       0.96      0.94      0.95        77
weighted avg       0.95      0.95      0.95        77
 

4. Confusion matrix 
 [[14  0  0  0  0  0]
 [ 0  9  0  0  0  0]
 [ 0  0  9  0  1  1]
 [ 0  0  0 10  0  1]
 [ 0  0  0  0 16  0]
 [ 0  0  0  1  0 15]] 

5. Accuracy 
 0.948051948051948 



In [39]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1:]
oversample = SMOTE()
X_ROS, y_ROS = oversample.fit_resample(X,y)
y_encode = LabelEncoder().fit_transform(y_ROS)
x_train,x_test,y_train,y_test = train_test_split(X_ROS, y_encode,test_size=0.20)
cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)

svclassifier = SVC()
# svclassifier.fit(x_train,y_train )
# pred_probs = svclassifier.predict(x_test)
# accuracy = accuracy_score(y_test,pred_probs)
# print("Test accuracy using SVM",accuracy)
# train_probs = svclassifier.predict(x_train)
# accuracy = accuracy_score(y_train,train_probs)
# print("Train accuracy using SVM",accuracy)
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','liner','poly']}
svmgrid = GridSearchCV(svclassifier, param_grid, refit = True, cv= cv) 
svmgrid.fit(x_train, y_train)
svm_pred_probs = svmgrid.predict(x_test)
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, svm_pred_probs, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, svm_pred_probs, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, svm_pred_probs)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, svm_pred_probs)))
print('5. Accuracy \n {} \n'.format(accuracy_score(y_test, svm_pred_probs)))

filename = 'respiratory_without_noise_results\\svm_model.pkl'
with open(filename, 'wb') as file:  
    pickle.dump(svmgrid, file)


1. The F-1 score of the model 0.9192229695413058

2. The recall score of the model 0.9203081232492997

3. Classification report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.78      1.00      0.88        14
           2       1.00      0.86      0.92        14
           3       1.00      0.76      0.87        17
           4       0.82      1.00      0.90        14
           5       1.00      0.90      0.95        10

    accuracy                           0.91        77
   macro avg       0.93      0.92      0.92        77
weighted avg       0.93      0.91      0.91        77
 

4. Confusion matrix 
 [[ 8  0  0  0  0  0]
 [ 0 14  0  0  0  0]
 [ 0  0 12  0  2  0]
 [ 0  3  0 13  1  0]
 [ 0  0  0  0 14  0]
 [ 0  1  0  0  0  9]] 

5. Accuracy 
 0.9090909090909091 



In [40]:
X = df.iloc[:, :-1]
y = df.iloc[:,-1:]
oversample = SMOTE()
X_ROS, y_ROS = oversample.fit_resample(X,y)
y_encode = LabelEncoder().fit_transform(y_ROS)
cv = StratifiedKFold(n_splits = 10, shuffle=True, random_state = 42)

x_train, x_test, y_train, y_test = train_test_split(X_ROS, y_encode, test_size=0.20)
model = XGBClassifier()

para_grid = {
                  'n_estimators': [2,5,10,15],
                   'max_depth':[1,3,10,20],
                   'learning_rate ':[0.001, 0.010, 0.100, 0.500],
                   'gamma' : [0, 0.5, 2, 10],
                   'subsample ':[1.0,0.9,0.5,0.1],
                   'min_child_weight ':[0.01, 0.05, 0.25, 0.75]

            }
# param_grid = dict(scale_pos_weight=weights)
XGboost = GridSearchCV(estimator=model, param_grid=para_grid, n_jobs=-1, cv=cv)
XGboost.fit(x_train, y_train)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
train_pred = XGboost.predict(x_test)
print('1. The F-1 score of the model {}\n'.format(f1_score(y_test, train_pred, average='macro')))
print('2. The recall score of the model {}\n'.format(recall_score(y_test, train_pred, average='macro')))
print('3. Classification report \n {} \n'.format(classification_report(y_test, train_pred)))
print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_test, train_pred)))
print('5. Accuracy \n {} \n'.format(accuracy_score(y_test, train_pred)))

filename = 'respiratory_without_noise_results\\xgboost_model.pkl'
with open(filename, 'wb') as file:  
    pickle.dump(XGboost, file)


[20:31:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { learning_rate , min_child_weight , subsample  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


1. The F-1 score of the model 0.8687927126636804

2. The recall score of the model 0.870379946850535

3. Classification report 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       0.88      0.93      0.90        15
           2       0.94      0.88      0.91        17
           3       0.71      0.71      0.71         7
           4       0.85      0.85      0.85        13
           5       0.92      0.85      0.88        13

    accuracy                           0.88        77
   macro avg  

In [41]:
norm_audios_feat = []
audio_path1 = "F:/dataset/Respiratory_Sound_Database/Respiratory_Sound_Database/clean1/101_1b1_Al_sc_Meditron.wav"
y,sr = librosa.load(audio_path1, sr=None)    #y = time-series array , sr = sampling rate
feature_vector = get_feature_vector(audio_path1, y, sr)
norm_audios_feat.append(feature_vector)
columns = ['id','chroma_stft', 'spectral_centroid','spectral_bandwidth','spectral_rolloff','rmse','zero_crossing_rate']
test_df = pd.DataFrame(norm_audios_feat,columns=columns)

# mfcc feature extraction 
id1 = []
index = ((audio_path1.split("/")[-1]).split("_")[0])
id1.append(index)
header = ['id']
for i in range(1,14):
    header.append(f' mfcc{i}')
mfcc_fea = []
mfcc_features = librosa.feature.mfcc(y,sr,n_mfcc=13)
mfcc = mfcc_features.T.tolist()
mfcc_1 = np.mean(mfcc, axis=0).tolist()
mfcc_fea.append(id1 + mfcc_1)
test_df1 = pd.DataFrame(mfcc_fea, columns=header)
test_df = test_df.merge(test_df1,on="id")
test_df


,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,101,0.790317,100.76941,421.074708,100.326977,0.057676,0.002898,-515.698943,113.96059,71.166173,42.36943,33.501165,28.330211,20.508287,16.753931,18.437936,16.671231,9.041621,3.930885,6.298694


In [42]:
target_name = ['Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy',
       'Pneumonia', 'URTI']

In [43]:
y_pred = rfgrid.predict(test_df)
y_classes = y_pred.argmax()
target_name[y_classes]

'Bronchiectasis'

In [44]:
y_pred = knngrid.predict(test_df)
y_classes = y_pred.argmax()
target_name[y_classes]

'Bronchiectasis'